# Creating a web scraper for Zillow.com
This scraper will search all of Zillow's listings for both recently sold and currently available single family homes in a specific state/county.

## Helpful videos:
https://www.youtube.com/watch?v=dRcvJRmqFHQ&ab_channel=WebScrapingwithAndy

https://www.youtube.com/watch?v=a3Cuq2csLWk

## Helpful websites:
curl.trillworks.com

In [75]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import numpy as np
import regex as re
import time
import json

In [76]:
# Initial variables
page = 0
county = 'brevard'
state = 'fl'
url = 'https://www.zillow.com/'

In [77]:
headers = {
    'authority': 'www.zillow.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'referer': 'https://www.zillow.com/',
    'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
    }

# response = requests.get(url=url, params=params, headers=headers)
response = requests.get(url=url, headers=headers)

In [78]:
# while response.status_code == 200:
while page <= 1:
    page += 1

    url = 'https://www.zillow.com/%s-county-%s/' %(county, state)
    
    params = {
    'searchQueryState': '{"pagination":{"currentPage":%s},"isMapVisible":false,"mapBounds":{"west":-81.67492903125,"east":-79.69738996875,"south":27.568978152602924,"north":29.04158767448029},"mapZoom":9,"usersSearchTerm":"Brevard County, FL","regionSelection":[{"regionId":1556,"regionType":4}],"filterState":{"sort":{"value":"globalrelevanceex"},"ah":{"value":true}},"isListVisible":true}' %page
    }

    response = requests.get(url=url, params=params, headers=headers)
    time.sleep(2)

    content = BeautifulSoup(response.text)

    deck = content.find('ul', {'class':'List-c11n-8-84-3__sc-1smrmqp-0 StyledSearchListWrapper-srp__sc-1ieen0c-0 doa-doM gKnRas photo-cards photo-cards_extra-attribution'})

    for card in deck.contents: # currently does not seem get all cards on page 1... missing a lot...
        script = card.find('script', {'type':'application/ld+json'}) # Excluding any advertisement cards. Script = None on ad cards.
        if script:
            script_json = json.loads(script.contents[0])
            print(script_json)


{'@type': 'SingleFamilyResidence', '@context': 'http://schema.org', 'name': '4945 Laguna Vista Dr, Melbourne, FL 32934', 'floorSize': {'@type': 'QuantitativeValue', '@context': 'http://schema.org', 'value': '2,234'}, 'address': {'@type': 'PostalAddress', '@context': 'http://schema.org', 'streetAddress': '4945 Laguna Vista Dr', 'addressLocality': 'Melbourne', 'addressRegion': 'FL', 'postalCode': '32934'}, 'geo': {'@type': 'GeoCoordinates', '@context': 'http://schema.org', 'latitude': 28.143036, 'longitude': -80.708885}, 'url': 'https://www.zillow.com/homedetails/4945-Laguna-Vista-Dr-Melbourne-FL-32934/43454629_zpid/'}
{'@type': 'SingleFamilyResidence', '@context': 'http://schema.org', 'name': '6425 Stillwater Ave, Cocoa, FL 32927', 'floorSize': {'@type': 'QuantitativeValue', '@context': 'http://schema.org', 'value': '1,984'}, 'address': {'@type': 'PostalAddress', '@context': 'http://schema.org', 'streetAddress': '6425 Stillwater Ave', 'addressLocality': 'Cocoa', 'addressRegion': 'FL', '

Run request on first page
get all child elements from cards on first page
    from child elements is a "homedetails" url for all the specific data
loop through all homedetails urls on first page as second request set
    scrape necessary data from homedetails page into data frame
run request on second, third, ..., nth page